<a href="https://colab.research.google.com/github/jaybakky2013/-Tableau-Data-analtyics/blob/main/STANCE_ASSIGNMENT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from fastai import *
from fastai.text import *
from pathlib import Path
import pandas as pd
import numpy as np
import re

In [ ]:

from google.colab import files
uploaded = files.upload()

In [ ]:
#Preprocessing Function
def clean_ascii(text):
    # function to remove non-ASCII chars from data
    return ''.join(i for i in text if ord(i) < 128)

In [ ]:
#Reading the data
trainfile='semeval2016-task6-trainingdata.txt'
testfile = 'SemEval2016-Task6-subtaskA-testdata.txt'
train_orig = pd.read_csv(trainfile, delimiter='\t', header=0, encoding = "latin-1")

In [ ]:

# Create directory
dirName = 'tempDi'
os.mkdir(dirName)
print("Directory " , dirName ,  " Created ") 


In [ ]:
# Plot value counts for each stance in the dataset
train_orig['Stance'].value_counts().plot.bar(rot=30);

In [ ]:
print("Tweet Topics:\n")
print("------------------------------------------")
for topic in train_orig['Target'].unique():
    print(topic)

In [ ]:
#Removing non-ascii characters
train_orig['Tweet'] = train_orig['Tweet'].apply(clean_ascii)

In [ ]:
train_orig.head()

In [ ]:
train = pd.concat([train_orig['Stance'], train_orig['Tweet']], axis=1)
train.tail()

In [ ]:
# Write train to csv
train.to_csv('tempDi/train.csv', index=False, header=False)

In [ ]:
data_lm = TextLMDataBunch.from_csv('tempDi', 'train.csv', min_freq=1)

In [ ]:
# Save the language and classifier model data for re-use
data_lm.save()

In [ ]:
data_lm.show_batch()

In [ ]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.1)

In [ ]:
print(learn.model)


In [ ]:
print(learn.summary())

In [ ]:
#finding the optimum learning rate
learn.lr_find(start_lr=1e-8, end_lr=1e2)
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(cyc_len=1, max_lr=2e-3, moms=(0.8, 0.7))

In [ ]:
#Unfreezing the entire model
learn.unfreeze()
learn.fit_one_cycle(cyc_len=3, max_lr=1e-3, moms=(0.8, 0.7))

In [ ]:
# Save the fine-tuned encoder
learn.save_encoder('ft_enc')

In [ ]:

for item in train_orig.Target.unique():
    print(item)

In [ ]:
topic = "Feminist Movement"
# Get only those tweets that pertain to a single topic in the training data
train_topic = train.loc[train_orig['Target'] == topic]

# Write train to csv
train_topic.to_csv('tempDi/train_topic.csv', index=False, header=False)
# Classifier model data
data_clas = TextClasDataBunch.from_csv('tempDi','train_topic.csv', vocab=data_lm.train_ds.vocab,
                                       min_freq=1, bs=32)
data_clas.save()
learn = text_classifier_learner( data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('ft_enc')
learn.freeze()
learn.lr_find(start_lr=1e-8, end_lr=1e2)
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(cyc_len=1, max_lr=1e-3, moms=(0.8, 0.7))

In [ ]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(1e-5,5e-3), moms=(0.8,0.7))

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-5,1e-1), moms=(0.8,0.7))

In [ ]:
# get predictions
preds, targets = learn.get_preds()
predictions = np.argmax(preds, axis=1)
pd.crosstab(predictions, targets)


In [ ]:
topic1_accuracy=accuracy(preds,targets)
print("Accuracy for Topic 1 (Feminism):\n")
print("------------------------------------------")
print (topic1_accuracy)

In [ ]:
test = pd.read_csv(testfile, delimiter='\t', header=0, encoding = "latin-1")
test = test.drop(['ID'], axis=1)
# test['Tweet'] = test['Tweet'].apply(clean_ascii)
test.head()

In [ ]:
test_pred = test[['Target', 'Tweet']]
test_pred = test_pred.loc[test_pred['Target'] == topic]
test_pred.tail()

In [ ]:
test_pred['Stance'] = test_pred['Tweet'].apply(lambda row: str(learn.predict(row)[0]))


In [ ]:
test_pred.tail()

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
from google.colab import files
test_pred.to_csv('feminismpred.txt', sep='\t', index=True,
                 header=['Target', 'Tweet', 'Stance'], index_label='ID')



In [ ]:

files.download('feminismpred.txt')

In [ ]:
topic2="Hillary Clinton"
train_topic2 = train.loc[train_orig['Target'] == topic2]


# Write train to csv
train_topic2.to_csv('tempDi/train_topic2.csv', index=False, header=False)
# Classifier model data
data_clas2 = TextClasDataBunch.from_csv('tempDi','train_topic2.csv', vocab=data_lm.train_ds.vocab,
                                       min_freq=1, bs=32)
data_clas2.save()
learn2 = text_classifier_learner( data_clas2,AWD_LSTM, drop_mult=0.5)
learn2.load_encoder('ft_enc')
learn2.freeze()
learn2.lr_find(start_lr=1e-8, end_lr=1e2)
learn2.recorder.plot()

In [ ]:
learn2.fit_one_cycle(cyc_len=1, max_lr=1e-3, moms=(0.8, 0.7))
learn2.freeze_to(-3)
learn2.fit_one_cycle(1, slice(1e-5,5e-3), moms=(0.8,0.7))
learn2.unfreeze()
learn2.fit_one_cycle(2, slice(1e-5,1e-1), moms=(0.8,0.7))


In [ ]:
# get predictions
preds2, targets2 = learn2.get_preds()
predictions2 = np.argmax(preds2, axis=1)
pd.crosstab(predictions2, targets2)
topic2_accuracy=accuracy(preds2,targets2)
print("Accuracy for Topic 2 (Hilary Clinton):\n")
print("------------------------------------------")
print (topic2_accuracy)

In [ ]:
test_pred2 = test[['Target', 'Tweet']]
test_pred2 = test_pred2.loc[test_pred2['Target'] == topic2]
test_pred2.tail()

In [ ]:
test_pred2['Stance'] = test_pred2['Tweet'].apply(lambda row: str(learn2.predict(row)[0]))

In [ ]:
test_pred2.tail()

In [ ]:
test_pred2.to_csv('h_clintonpred.txt', sep='\t', index=True,
                 header=['Target', 'Tweet', 'Stance'], index_label='ID')

In [ ]:

files.download('h_clintonpred.txt')

In [ ]:
topic3="Climate Change is a Real Concern"
train_topic3 = train.loc[train_orig['Target'] == topic3]


# Write train to csv
train_topic3.to_csv('tempDi/train_topic3.csv', index=False, header=False)
# Classifier model data
data_clas3 = TextClasDataBunch.from_csv('tempDi','train_topic3.csv', vocab=data_lm.train_ds.vocab,
                                       min_freq=1, bs=32)
data_clas3.save()
learn3 = text_classifier_learner( data_clas3,AWD_LSTM, drop_mult=0.5)
learn3.load_encoder('ft_enc')
learn3.freeze()
learn3.lr_find(start_lr=1e-8, end_lr=1e2)
learn3.recorder.plot()

In [ ]:
learn3.fit_one_cycle(cyc_len=1, max_lr=1e-3, moms=(0.8, 0.7))
learn3.freeze_to(-3)
learn3.fit_one_cycle(1, slice(1e-5,5e-3), moms=(0.8,0.7))
learn3.unfreeze()
learn3.fit_one_cycle(2, slice(1e-5,1e-1), moms=(0.8,0.7))


In [ ]:
# get predictions
preds3, targets3 = learn3.get_preds()
predictions3 = np.argmax(preds3, axis=1)
print(pd.crosstab(predictions3, targets3))
topic3_accuracy=accuracy(preds3,targets3)
print("Accuracy for Topic 3 (Climate Change is a Real Concern):\n")
print("------------------------------------------")
print (topic3_accuracy)

In [ ]:
test_pred3 = test[['Target', 'Tweet']]
test_pred3 = test_pred3.loc[test_pred3['Target'] == topic3]
test_pred3.tail()

In [ ]:
test_pred3['Stance'] = test_pred3['Tweet'].apply(lambda row: str(learn3.predict(row)[0]))
test_pred3.tail()

In [ ]:
test_pred3.to_csv('climatepred.txt', sep='\t', index=True,
                 header=['Target', 'Tweet', 'Stance'], index_label='ID')

In [ ]:

files.download('climatepred.txt')

In [ ]:
topic4="Atheism"
train_topic4 = train.loc[train_orig['Target'] == topic4]


# Write train to csv
train_topic4.to_csv('tempDi/train_topic4.csv', index=False, header=False)
# Classifier model data
data_clas4 = TextClasDataBunch.from_csv('tempDi','train_topic4.csv', vocab=data_lm.train_ds.vocab,
                                       min_freq=1, bs=32)
data_clas4.save()
learn4 = text_classifier_learner( data_clas4,AWD_LSTM, drop_mult=0.5)
learn4.load_encoder('ft_enc')
learn4.freeze()
learn4.lr_find(start_lr=1e-8, end_lr=1e2)
learn4.recorder.plot()

In [ ]:
learn4.fit_one_cycle(cyc_len=1, max_lr=1e-3, moms=(0.8, 0.7))
learn4.freeze_to(-3)
learn4.fit_one_cycle(1, slice(1e-5,5e-3), moms=(0.8,0.7))
learn4.unfreeze()
learn4.fit_one_cycle(2, slice(1e-5,1e-1), moms=(0.8,0.7))

In [ ]:
# get predictions
preds4, targets4 = learn4.get_preds()
predictions4 = np.argmax(preds4, axis=1)
print(pd.crosstab(predictions4, targets4))
topic4_accuracy=accuracy(preds4,targets4)
print("Accuracy for Topic 4 (Atheism):\n")
print("------------------------------------------")
print (topic4_accuracy)

In [ ]:
test_pred4 = test[['Target', 'Tweet']]
test_pred4 = test_pred4.loc[test_pred4['Target'] == topic4]
test_pred4.tail()

In [ ]:
test_pred4['Stance'] = test_pred4['Tweet'].apply(lambda row: str(learn4.predict(row)[0]))
test_pred4.tail()

In [ ]:
test_pred4.to_csv('atheismpred.txt', sep='\t', index=True,
                 header=['Target', 'Tweet', 'Stance'], index_label='ID')

In [ ]:

files.download('atheismpred.txt')

In [ ]:
topic5="Legalization of Abortion"
train_topic5 = train.loc[train_orig['Target'] == topic5]


# Write train to csv
train_topic5.to_csv('tempDi/train_topic5.csv', index=False, header=False)
# Classifier model data
data_clas5 = TextClasDataBunch.from_csv('tempDi','train_topic5.csv', vocab=data_lm.train_ds.vocab,
                                       min_freq=1, bs=32)
data_clas5.save()
learn5 = text_classifier_learner( data_clas5,AWD_LSTM, drop_mult=0.5)
learn5.load_encoder('ft_enc')
learn5.freeze()
learn5.lr_find(start_lr=1e-8, end_lr=1e2)
learn5.recorder.plot()

In [ ]:
learn5.fit_one_cycle(cyc_len=1, max_lr=1e-3, moms=(0.8, 0.7))
learn5.freeze_to(-3)
learn5.fit_one_cycle(1, slice(1e-5,5e-3), moms=(0.8,0.7))
learn5.unfreeze()
learn5.fit_one_cycle(2, slice(1e-5,1e-1), moms=(0.8,0.7))

In [ ]:
# get predictions
preds5, targets5 = learn5.get_preds()
predictions5 = np.argmax(preds5, axis=1)
print(pd.crosstab(predictions5, targets5))
topic5_accuracy=accuracy(preds5,targets5)
print("Accuracy for Topic 5 (Legalization of Abortion):\n")
print("------------------------------------------")
print (topic5_accuracy)

In [ ]:
test_pred5 = test[['Target', 'Tweet']]
test_pred5 = test_pred5.loc[test_pred5['Target'] == topic5]
test_pred5.tail()

In [ ]:
test_pred5['Stance'] = test_pred5['Tweet'].apply(lambda row: str(learn5.predict(row)[0]))
test_pred5.tail()

In [ ]:
test_pred5.to_csv('abortionpred.txt', sep='\t', index=True,
                 header=['Target', 'Tweet', 'Stance'], index_label='ID')

In [ ]:
files.download('abortionpred.txt')

In [ ]:
print("------------------------------------------")
print("Accuracy for Feminism:",topic1_accuracy)
print("------------------------------------------")
print("Accuracy for Hilary Clinton:",topic2_accuracy)
print("------------------------------------------")
print("Climate Change is a Real Concern:",topic3_accuracy)
print("------------------------------------------")
print("Atheism:",topic4_accuracy)
print("------------------------------------------")
print("Legalization of Abortion:",topic5_accuracy)
print("------------------------------------------")
